##1. Unzipping Folder

In [ ]:
!apt-get install -y unrar

In [ ]:
!unrar x /content/VanGogh.rar

##2. Install all Requirements

In [ ]:
!pip install fastapi uvicorn jinja2 pyngrok python-multipart dotenv

##3. Downloading and Loading Model(Stable Diffusion V-1.5) with LoRA Weights

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

pipe = StableDiffusionPipeline.from_pretrained(
    "sd-legacy/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    cache_dir="/root/.cache/huggingface"  # reuse cache
).to(device)

pipe.load_lora_weights(
    "/content/VanGogh/lora",
    weight_name="vangogh_lora.safetensors"
)

##3. Image function

In [ ]:
def generate_image(prompt):
  image = pipe(
      f"<vangogh> {prompt}",
      num_inference_steps=30,
      guidance_scale=7.5
  ).images[0]
  image.save('VanGogh/image/generated_image.png')


##4. FastAPI

In [ ]:
from fastapi import FastAPI, Request, Form
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.responses import RedirectResponse
from pathlib import Path
from uvicorn import Config, Server
from pyngrok import ngrok

##5. FastAPI Routes

In [ ]:
app = FastAPI()
templates = Jinja2Templates(directory='VanGogh/template')
app.mount('/static',StaticFiles(directory='VanGogh/image'),name='static')

path = Path('VanGogh/image/generated_image.png')

@app.get('/')
def home(request: Request,
         prompt: str|None = None):
  image_exist = path.exists()
  return templates.TemplateResponse(name='index.html',request=request,context={'image_exist':image_exist,'prompt':prompt})

@app.post('/')
def post_home(prompt = Form(...)):
  generate_image(prompt)
  return RedirectResponse(url=f'/?prompt={prompt}',status_code=303)

##6. Loading Token from .env

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='VanGogh/.env')

token = os.getenv('TOKEN')

##7. Making Server Live through ngrok Tunneling

In [ ]:
ngrok.kill()
ngrok.set_auth_token(token)

tunnel = ngrok.connect(8000)
print("Public URL:", tunnel.public_url)

config = Config(app)
server = Server(config=config)
await server.serve()

##8. Kill Tunnel after Execution

In [ ]:
ngrok.kill()